In [ ]:
# ======================================================
# FraudDetectPro: Two-Stage Hybrid Model Pipeline
# ======================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_recall_curve
import shap
import matplotlib.pyplot as plt
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# =====================
# Setup
# =====================
os.makedirs("../models", exist_ok=True)
os.makedirs("../visualizations", exist_ok=True)

print("="*65)
print("FRAUDDETECTPRO - TWO-STAGE HYBRID MODEL TRAINING")
print("="*65)

# =====================
# Load Processed Data
# =====================
print("\n📂 Loading preprocessed data...")
X_train = np.load("../data/processed/X_train.npy")
y_train = np.load("../data/processed/y_train.npy")
X_test = np.load("../data/processed/X_test.npy")
y_test = np.load("../data/processed/y_test.npy")

try:
    feature_names = np.load("../data/processed/feature_names.npy", allow_pickle=True)
    print(f"✓ Loaded {len(feature_names)} feature names")
except FileNotFoundError:
    print("⚠️  feature_names.npy not found. Using generic names.")
    feature_names = [f"Feature_{i}" for i in range(X_train.shape[1])]

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Fraud ratio in training: {y_train.mean() * 100:.2f}%")
print(f"Fraud ratio in test: {y_test.mean() * 100:.2f}%")

# ======================================================
# STAGE 1 — Neural Network Feature Extractor
# ======================================================
print("\n🧠 Building Neural Network Feature Extractor...")

input_dim = X_train.shape[1]

# Using Functional API 
inputs = Input(shape=(input_dim,), name='input_layer')
x = Dense(64, activation='relu', name='dense_1')(inputs)
x = Dropout(0.3, name='dropout_1')(x)
feature_layer = Dense(32, activation='relu', name='feature_layer')(x)  # ← Extract from here
x = Dropout(0.2, name='dropout_2')(feature_layer)
outputs = Dense(1, activation='sigmoid', name='output')(x)

# Build full model
nn_full = Model(inputs=inputs, outputs=outputs, name='nn_full')

nn_full.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(nn_full.summary())

# Train the full model
print("\n🏋️  Training full neural network...")
es = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

history = nn_full.fit(
    X_train, y_train,
    epochs=20,
    batch_size=256,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

# ✅ Now we can create the feature extractor safely
print("\n🔧 Creating feature extractor...")
feature_extractor = Model(
    inputs=nn_full.input,
    outputs=nn_full.get_layer('feature_layer').output,
    name='feature_extractor'
)

print(feature_extractor.summary())

# Extract features
print("\nExtracting features from neural network...")
X_train_nn = feature_extractor.predict(X_train, verbose=0)
X_test_nn = feature_extractor.predict(X_test, verbose=0)

print(f"✓ Original features: {X_train.shape[1]}")
print(f"✓ Extracted NN features: {X_train_nn.shape[1]}")

# Combine original + extracted features
X_train_hybrid = np.hstack((X_train, X_train_nn))
X_test_hybrid = np.hstack((X_test, X_test_nn))
print(f"✓ Hybrid feature set shape: {X_train_hybrid.shape}")

# ======================================================
# STAGE 2 — Ensemble Models on Hybrid Features
# ======================================================
print("\n🤖 Training ensemble models on hybrid features...")

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=(len(y_train) - y_train.sum()) / (y_train.sum() + 1e-9),
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1,
    use_label_encoder=False
)

lr = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Training Random Forest...")
rf.fit(X_train_hybrid, y_train)

print("Training XGBoost...")
xgb.fit(X_train_hybrid, y_train)

print("Training Logistic Regression...")
lr.fit(X_train_hybrid, y_train)

# ======================================================
# Ensemble Predictions
# ======================================================
def ensemble_predict(models, X, threshold=0.5):
    """Soft voting ensemble prediction"""
    probs = np.zeros(X.shape[0])
    for model in models:
        probs += model.predict_proba(X)[:, 1]
    probs /= len(models)
    preds = (probs >= threshold).astype(int)
    return preds, probs

# Find optimal threshold
print("\n🎯 Finding optimal threshold...")
models = [rf, xgb, lr]
_, y_prob_train = ensemble_predict(models, X_train_hybrid)

precision, recall, thresholds = precision_recall_curve(y_train, y_prob_train)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

# Handle edge case where no thresholds are returned
if len(thresholds) == 0:
    optimal_threshold = 0.5
    print("⚠️  No thresholds found; using default 0.5")
else:
    # Precision/recall arrays are 1 element longer than thresholds
    valid_f1 = f1_scores[:-1] if len(f1_scores) > len(thresholds) else f1_scores
    optimal_idx = np.argmax(valid_f1)
    optimal_threshold = thresholds[optimal_idx]
    print(f"✓ Optimal threshold: {optimal_threshold:.3f} (F1: {valid_f1[optimal_idx]:.3f})")

# ======================================================
# Test Set Evaluation
# ======================================================
print("\n📊 Evaluating on test set...")
y_pred, y_prob = ensemble_predict(models, X_test_hybrid, threshold=optimal_threshold)

print("\n" + "="*60)
print("ENSEMBLE MODEL EVALUATION (Hybrid Features)")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Legitimate', 'Fraud']))

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]:,}")
print(f"False Positives: {cm[0,1]:,}")
print(f"False Negatives: {cm[1,0]:,}")
print(f"True Positives:  {cm[1,1]:,}")

print(f"\n✅ F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"✅ ROC-AUC:  {roc_auc_score(y_test, y_prob):.4f}")

# Individual model performance
print("\n" + "="*60)
print("INDIVIDUAL MODEL PERFORMANCE")
print("="*60)
for name, model in [('Random Forest', rf), ('XGBoost', xgb), ('Logistic Regression', lr)]:
    y_pred_single = model.predict(X_test_hybrid)
    y_prob_single = model.predict_proba(X_test_hybrid)[:, 1]
    print(f"\n{name}:")
    print(f"  F1-Score: {f1_score(y_test, y_pred_single):.4f}")
    print(f"  ROC-AUC:  {roc_auc_score(y_test, y_prob_single):.4f}")

# ======================================================
# SHAP Explainability
# ======================================================
print("\n🔍 Generating SHAP explanations...")
sample_size = min(1000, X_test_hybrid.shape[0])
X_test_sample = X_test_hybrid[:sample_size]

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_test_sample)

# Combined feature names
nn_feature_count = X_train_nn.shape[1]
combined_feature_names = list(feature_names) + [f"NN_Feature_{i}" for i in range(nn_feature_count)]

# Summary plot
plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values,
    X_test_sample,
    feature_names=combined_feature_names,
    show=False
)
plt.tight_layout()
plt.savefig("../visualizations/shap_summary_hybrid.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ SHAP summary plot saved")

# Feature importance bar plot
plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values,
    X_test_sample,
    feature_names=combined_feature_names,
    plot_type="bar",
    show=False
)
plt.tight_layout()
plt.savefig("../visualizations/shap_importance_hybrid.png", dpi=300, bbox_inches='tight')
plt.close()
print("✓ SHAP importance plot saved")

# ======================================================
# Save All Models and Metadata
# ======================================================
print("\n💾 Saving models...")

# Save both full model and feature extractor
nn_full.save("../models/nn_full_model.h5")
feature_extractor.save("../models/nn_feature_extractor.h5")

joblib.dump(rf, "../models/rf_model.pkl")
joblib.dump(xgb, "../models/xgb_model.pkl")
joblib.dump(lr, "../models/lr_model.pkl")

metadata = {
    'feature_names': feature_names.tolist(),
    'optimal_threshold': float(optimal_threshold),
    'input_features': int(X_train.shape[1]),
    'nn_features': int(nn_feature_count),
    'hybrid_features': int(X_train_hybrid.shape[1]),
    'test_metrics': {
        'f1_score': float(f1_score(y_test, y_pred)),
        'roc_auc': float(roc_auc_score(y_test, y_prob)),
        'confusion_matrix': cm.tolist(),
        'true_negatives': int(cm[0,0]),
        'false_positives': int(cm[0,1]),
        'false_negatives': int(cm[1,0]),
        'true_positives': int(cm[1,1])
    }
}
joblib.dump(metadata, "../models/model_metadata.pkl")

print("\n" + "="*60)
print("✅ TWO-STAGE HYBRID MODEL TRAINING COMPLETE!")
print("="*60)
print("\nSaved files:")
print("  ✓ ../models/nn_full_model.h5")
print("  ✓ ../models/nn_feature_extractor.h5  ← For API")
print("  ✓ ../models/rf_model.pkl")
print("  ✓ ../models/xgb_model.pkl")
print("  ✓ ../models/lr_model.pkl")
print("  ✓ ../models/model_metadata.pkl")
print("  ✓ ../visualizations/shap_summary_hybrid.png")
print("  ✓ ../visualizations/shap_importance_hybrid.png")

print(f"\n📈 Final Performance Summary:")
print(f"   F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"   ROC-AUC:  {roc_auc_score(y_test, y_prob):.4f}")
print(f"   Threshold: {optimal_threshold:.3f}")

c:\Users\x390\github-classroom\TamaraChelagat\GRP-A-ISP-TamaraChelagat\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FRAUDDETECTPRO - TWO-STAGE HYBRID MODEL TRAINING

📂 Loading preprocessed data...
✓ Loaded 30 feature names

Training set: (454902, 30)
Test set: (56962, 30)
Fraud ratio in training: 50.00%
Fraud ratio in test: 0.17%

🧠 Building Neural Network Feature Extractor...


Model: "nn_full"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ feature_layer (Dense)           │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

...

 Total params: 4,064 (15.88 KB)

 Trainable params: 4,064 (15.88 KB)

 Non-trainable params: 0 (0.00 B)

None

Extracting features from neural network...
✓ Original features: 30
✓ Extracted NN features: 32
✓ Hybrid feature set shape: (454902, 62)

🤖 Training ensemble models on hybrid features...
Training Random Forest...
Training XGBoost...
Training Logistic Regression...

🎯 Finding optimal threshold...
✓ Optimal threshold: 0.950 (F1: 1.000)

📊 Evaluating on test set...

ENSEMBLE MODEL EVALUATION (Hybrid Features)
              precision    recall  f1-score   support

  Legitimate       1.00      1.00      1.00     56864
       Fraud       0.86      0.79      0.82        98

    accuracy                           1.00     56962
   macro avg       0.93      0.89      0.91     56962
weighted avg       1.00      1.00      1.00     56962


Confusion Matrix:
[[56851    13]
 [   21    77]]

True Negatives:  56,851
False Positives: 13
False Negatives: 21
True Positives:  77

✅ F1-Score: 0.8191
✅ ROC-AUC:  0.9583

INDIVIDUAL MODEL PERFORMANCE

Random Forest:
  F1-Score: 0.7960
  ROC-AUC:  0.9335


✓ SHAP importance plot saved

💾 Saving models...

✅ TWO-STAGE HYBRID MODEL TRAINING COMPLETE!

Saved files:
  ✓ ../models/nn_full_model.h5
  ✓ ../models/nn_feature_extractor.h5  ← For API
  ✓ ../models/rf_model.pkl
  ✓ ../models/xgb_model.pkl
  ✓ ../models/lr_model.pkl
  ✓ ../models/model_metadata.pkl
  ✓ ../visualizations/shap_summary_hybrid.png
  ✓ ../visualizations/shap_importance_hybrid.png

📈 Final Performance Summary:
   F1-Score: 0.8191
   ROC-AUC:  0.9583
   Threshold: 0.950
